<a href="https://colab.research.google.com/github/DayaSribala/PCA/blob/main/Deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Comment locally for local executions
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
# Comment locally for local executions
!cd "/content/drive/My Drive/"

In [5]:
# Comment locally for local executions
!cd "/content/drive/My Drive/dfdc_train_part_31"

Import Libraries

In [6]:
!pip install -U --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [143]:
import sys
import sklearn
import cv2
import os
import glob
import pathlib
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib import pyplot as plt

In [144]:
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Reshape  # Import Reshape layer
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

In [145]:
tf.test.is_gpu_available()

False

In [146]:
tf.config.list_physical_devices('GPU')

[]

In [147]:
tf.__version__

'2.16.1'

Load Dataset


In [148]:
train_path = '/content/drive/My Drive/dfdc_train_part_31/Train'
test_path = '/content/drive/My Drive/dfdc_train_part_31/Test'

In [149]:
#function to get count of images
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
      return count

In [150]:
train_samples =get_files(train_path)
num_classes=len(glob.glob(train_path+"/*"))
test_samples=get_files(test_path)
print(num_classes,"Classes")
print(train_samples,"Train")
print(test_samples,"Test")

2 Classes
1 Train
1 Test


In [151]:
# Load video frames from MP4 files
def load_video_frames(video_file):
    cap = cv2.VideoCapture(video_file)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return np.array(frames)

In [152]:
import pathlib

train_path = '/content/drive/My Drive/dfdc_train_part_31/Train'  # Replace with your train directory path
train_path = pathlib.Path(train_path)  # Create a Path object

train_videos = []
train_labels = []
for file in train_path.glob('*.mp4'):
    frames = load_video_frames(str(file))
    label = 1 if 'eal' in str(file) else 0

# Load training data
train_videos = []
train_labels = []
for file in train_path.glob('*.mp4'):
    frames = load_video_frames(str(file))
    label = 1 if 'real' in str(file) else 0
    train_videos.append(frames)
    train_labels.append(label)

In [153]:
import pathlib

test_path = '/content/drive/My Drive/dfdc_train_part_31/Test'  # Replace with your test directory path
test_path = pathlib.Path(test_path)  # Create a Path object

test_videos = []
test_labels = []
for file in test_path.glob('*.mp4'):
    frames = load_video_frames(str(file))
    label = 1 if 'eal' in str(file) else 0

# Load testing data
test_videos = []
test_labels = []
for file in test_path.glob('*.mp4'):
    frames = load_video_frames(str(file))
    label = 1 if 'real' in str(file) else 0
    test_videos.append(frames)
    test_labels.append(label)

In [154]:
# Preprocess video frames
def preprocess_frames(frames):
    frames = frames / 255.0
    frames = frames.reshape((frames.shape[0], frames.shape[1], frames.shape[2], 3))
    return frames

In [155]:
train_videos = [preprocess_frames(frames) for frames in train_videos]
test_videos = [preprocess_frames(frames) for frames in test_videos]

In [156]:
model = Sequential()
model.add(Conv3D(32, (3, 3, 3), activation='relu', input_shape=(16, 224, 224, 3), padding='same'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Conv3D(128, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Reshape((16, 8)))  # Reshape to (timesteps, features)
model.add(LSTM(64, return_sequences=False))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_40 (Conv3D)                   │ (None, 16, 224, 224, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_39 (MaxPooling3D)      │ (None, 8, 112, 112, 32)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_41 (Conv3D)                   │ (None, 8, 112, 112, 64)     │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_40 (MaxPooling3D)      │ (None, 4, 56, 56, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_42 (Conv3D)                   │ (None, 4, 56, 56, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_41 (MaxPooling3D)      │ (None, 2, 28, 28, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_13 (Flatten)                 │ (None, 200704)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 128)                 │      25,690,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_7 (Reshape)                  │ (None, 16, 8)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 64)                  │          18,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,988,354 (99.14 MB)

 Trainable params: 25,988,354 (99.14 MB)

 Non-trainable params: 0 (0.00 B)

In [157]:
model_layers = [ layer.name for layer in model.layers]
print('layer name : ' ,model_layers)

layer name :  ['conv3d_40', 'max_pooling3d_39', 'conv3d_41', 'max_pooling3d_40', 'conv3d_42', 'max_pooling3d_41', 'flatten_13', 'dense_21', 'reshape_7', 'lstm_9', 'dense_22']


In [158]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [159]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)